In [20]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, TimeDistributed
from keras.layers import Concatenate, Flatten
from keras.layers import GRU, Conv2D, MaxPooling2D
from keras.layers import Input, Reshape, Dot, Add
from keras.models import Model
from keras.optimizers import Adam
from keras.optimizers import RMSprop
# from keras.utils.vis_utils import plot_model
import keras
import keras.backend as K
from data_handle import *
from gensim_wrapper import *
from utils import *
import gensim

In [2]:
def conv_model(n_input, n_output, n_enc_units, n_dec_units):
    root_word_input = Input(shape=(13, 309, 1), name="root_word_input")
    
    x = Conv2D(16, (3, 3), padding='same', activation='relu')(root_word_input)
    x = MaxPooling2D(2, 2)(x)
    X = Dropout(.2)(x)
    x = Conv2D(8, (3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D(2, 2)(x)
    x = Flatten()(x)
#     x = Dense(300, activation='relu')(x)
    X = Dropout(.2)(x)
    state_h = Dense(n_dec_units, activation='linear')(x)
    
    decoder_inputs = Input(shape=(None, 309), name="target_word_input")
    decoder_gru = GRU(n_dec_units, return_sequences=True, return_state=True, name="decoder_gru")
    decoder_outputs, _= decoder_gru(decoder_inputs, initial_state=state_h)
    
    decoder_dense = Dense(309, activation='softmax', name="train_output")
    decoder_outputs = decoder_dense(decoder_outputs)
    
    model = Model([root_word_input, decoder_inputs], decoder_outputs)
    encoder_model = Model(root_word_input, state_h)
    
    decoder_state_input_h = Input(shape=(n_dec_units,))
    decoder_outputs, state_h= decoder_gru(decoder_inputs, initial_state=decoder_state_input_h)

    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs, decoder_state_input_h], [decoder_outputs, state_h])

    return model, encoder_model, decoder_model


In [41]:
def conv_model_multi(n_chars, n_consonant, n_vowels, n_units):
    root_word_input = Input(shape=(n_chars, (n_consonant + n_vowels), 1), name="root_word_input")
    
    x = Conv2D(16, (3, 3), padding='same', activation='relu')(root_word_input)
    x = MaxPooling2D(2, 2)(x)
    X = Dropout(.2)(x)
    x = Conv2D(8, (3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D(2, 2)(x)
    x = Flatten()(x)
#     x = Dense(300, activation='relu')(x)
#     X = Dropout(.2)(x)
    state_h = Dense(n_units, activation='linear')(x)
    
    consonant_decoder_inputs = Input(shape=(None, n_consonant), name="target_consonant")
    consonant_decoder_gru = GRU(n_units, return_sequences=True, return_state=True, name="consonant_decoder_gru")
    consonant_decoder_outputs, _= consonant_decoder_gru(consonant_decoder_inputs, initial_state=state_h)
    
    vowel_decoder_inputs = Input(shape=(None, n_vowels), name="vowel_input")
    vowel_decoder_gru = GRU(n_units, return_sequences=True, return_state=True, name="vowl_decoder_gru")
    vowel_decoder_outputs, _= vowel_decoder_gru(vowel_decoder_inputs, initial_state=state_h)
    print(vowel_decoder_outputs.shape, consonant_decoder_outputs.shape)
    x = Concatenate(axis=1)([vowel_decoder_outputs, consonant_decoder_outputs])
    print(x.shape)
    consonant_decoder_dense = Dense(n_consonant, activation='softmax', name="consonant_output")
    consonant_decoder_outputs = consonant_decoder_dense(x)
    
    vowel_decoder_dense = Dense(n_vowels, activation='softmax', name="vowel_output")
    vowel_decoder_outputs = vowel_decoder_dense(x)
    
    main_model = Model([root_word_input, consonant_decoder_inputs, vowel_decoder_inputs], [consonant_decoder_outputs, vowel_decoder_outputs])
    
    encoder_model = Model(root_word_input, state_h)
    
    decoder_state_input_h = Input(shape=(n_units,))
    
    consonant_decoder_outputs, state_h= consonant_decoder_gru(consonant_decoder_inputs, initial_state=decoder_state_input_h)
    consonant_decoder_outputs = consonant_decoder_dense(consonant_decoder_outputs)
    
    vowel_decoder_outputs, state_h= vowel_decoder_gru(vowel_decoder_inputs, initial_state=decoder_state_input_h)
    vowel_decoder_outputs = vowel_decoder_dense(vowel_decoder_outputs)
    
    decoder_model = Model([consonant_decoder_inputs, vowel_decoder_inputs, decoder_state_input_h], [consonant_decoder_outputs, vowel_decoder_outputs, state_h])

    return main_model, encoder_model, decoder_model


In [4]:
def conv_model_multi_v2(n_chars, n_char_class, n_consonant, n_vowels, n_units):
    root_word_input = Input(shape=(n_chars, (n_consonant + n_vowels), 1), name="root_word_input")
    
    x = Conv2D(16, (3, 3), padding='same', activation='relu')(root_word_input)
    x = MaxPooling2D(2, 2)(x)
    X = Dropout(.2)(x)
    x = Conv2D(8, (3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D(2, 2)(x)
    x = Flatten()(x)
#     x = Dense(300, activation='relu')(x)
#     X = Dropout(.2)(x)
    state_h = Dense(n_units, activation='linear')(x)
    
    consonant_decoder_inputs = Input(shape=(None, n_consonant), name="target_consonant")
    consonant_decoder_gru = GRU(n_units, return_sequences=True, return_state=True, name="consonant_decoder_gru")
    consonant_decoder_outputs, _= consonant_decoder_gru(consonant_decoder_inputs, initial_state=state_h)
    
    vowel_decoder_inputs = Input(shape=(None, n_vowels), name="vowel_input")
    vowel_decoder_gru = GRU(n_units, return_sequences=True, return_state=True, name="vowl_decoder_gru")
    vowel_decoder_outputs, _= vowel_decoder_gru(vowel_decoder_inputs, initial_state=state_h)
    
    decoders_outputs = Concatenate(axis=1)([consonant_decoder_outputs, vowel_decoder_outputs])
  
    decoder_dense = Dense(n_char_class, activation='softmax', name="decoder_output")
    decoders_outputs = decoder_dense(decoders_outputs)
    
    main_model = Model([root_word_input, consonant_decoder_inputs, vowel_decoder_inputs], decoders_outputs)
    encoder_model = Model(root_word_input, state_h)
    
    decoder_state_input_h = Input(shape=(n_units,))
    
    consonant_decoder_outputs, state_h= consonant_decoder_gru(consonant_decoder_inputs, initial_state=decoder_state_input_h)
    vowel_decoder_outputs, state_h= vowel_decoder_gru(vowel_decoder_inputs, initial_state=decoder_state_input_h)
    
    decoders_outputs = Concatenate(axis=1)([consonant_decoder_outputs, vowel_decoder_outputs])
    decoders_outputs = decoder_dense(decoders_outputs)
    
    decoder_model = Model([consonant_decoder_inputs, vowel_decoder_inputs, decoder_state_input_h], [decoders_outputs, state_h])
    
    return main_model, encoder_model, decoder_model


In [5]:
def conv_model2(n_input, n_output, n_enc_units, n_dec_units):
    root_word_input = Input(shape=(13, 309, 1), name="root_word_input")
    word_feature = Input(shape=(128,), name="word_feature")
    
    x = Conv2D(16, (3, 3), padding='same', activation='relu')(root_word_input)
    x = MaxPooling2D(2, 2)(x)
    X = Dropout(.2)(x)
    x = Conv2D(8, (3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D(2, 2)(x)
    x = Flatten()(x)
    x = Dense(128, activation='linear')(x)
#     X = Dropout(.2)(x)
    x = Add()([x, word_feature])
    state_h = Dense(n_dec_units, activation='linear')(x)
    
    decoder_inputs = Input(shape=(None, 309), name="target_word_input")
    decoder_gru = GRU(n_dec_units, return_sequences=True, return_state=True, name="decoder_gru")
    decoder_outputs, _= decoder_gru(decoder_inputs, initial_state=state_h)
    
    decoder_dense = Dense(309, activation='softmax', name="train_output")
    decoder_outputs = decoder_dense(decoder_outputs)
    
    model = Model([root_word_input, decoder_inputs, word_feature], decoder_outputs)
    encoder_model = Model([root_word_input, word_feature], state_h)
    
    decoder_state_input_h = Input(shape=(n_dec_units,))
    decoder_outputs, state_h= decoder_gru(decoder_inputs, initial_state=decoder_state_input_h)

    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs, decoder_state_input_h], [decoder_outputs, state_h])

    return model, encoder_model, decoder_model

In [6]:
def decode_sequence(model, int2char, state):
    target_seq = np.zeros([1, 1, 309])
    target_seq[0, 0, char2int['&']] = 1
    decoded_chars = []
    for i in range(13):
        target_seq, state = model.predict([target_seq, state])
        index = np.argmax(target_seq.flatten())
        char = int2char[index]
        decoded_chars += [char]
        
#         target_seq = np.zeros([1, 1, 309])
#         target_seq[0, 0, index] = 1
    return decoded_chars


In [34]:
def one_hot_sep(con, con_max, vow, vow_max):
    con_vec = np.zeros((con_max, ))
    con_vec[con] = 1
    vow_vec = np.zeros((vow_max, ))
    vow_vec[vow] = 1
    return con_vec, vow_vec
    
def decode_multi_sequence(model, char2tup, tup2char, state, n_consonant, n_vowels):
    con, vow = char2tup['&']
    con_vec, vow_vec = one_hot_sep(con, n_consonant, vow, n_vowels) 
    con_vec = con_vec.reshape((1, 1, -1))
    vow_vec = vow_vec.reshape((1, 1, -1))
#     target_seq = np.concatenate([con_vec, vow_vec])
    decoded_chars = []
    for i in range(13):
        con_vec, vow_vec, state = model.predict([con_vec, vow_vec, state])
#         target_seq = np.concatenate([con_vec, vow_vec])
        new_con_vec = np.zeros_like(con_vec)
        new_con_vec[0, 0, np.argmax(con_vec[0, 0, :])] = 1
        new_vow_vec = np.zeros_like(vow_vec)
        new_vow_vec[0, 0, np.argmax(vow_vec[0, 0, :])] = 1
        con_vec, vow_vec = new_con_vec, new_vow_vec
        name = "{0}-{1}".format(np.argmax(con_vec[0, 0, :]), np.argmax(vow_vec[0, 0, :]))
        try:
            char = tup2char[name]
        except:
            char = ' '
        decoded_chars += [char]
    
    return decoded_chars

In [8]:
def embedding_model(input_size, output_size, embed_size):
    context_word = Input(shape=(input_size,), name="context_word")
    x = Dense(256, activation='relu')(context_word)
    embeding = Dense(embed_size, activation='tanh')(x)
    target_word = Dense(output_size, activation='relu')(embeding)
    model = Model(context_word, target_word)
    em_model = Model(context_word, embeding)
    return model, em_model

In [9]:
def cosine_loss(yTrue, yPred):
    loss = K.sum(K.square(yTrue - yPred))
    return loss

In [10]:
def embedding_model2(input_size, output_size, embed_size):
    context_word = Input(shape=(input_size,), name="context_word")
    target_word = Input(shape=(input_size,), name="target_word")
    
    layer1 = Dense(200, activation='tanh')
    layer2 = Dense(200, activation='tanh')
    
    x = layer1(context_word)
    y = layer1(target_word)
#     y = layer2(y)
    cosine_sim = Dot(normalize=True, axes=1)([x, y])
#     z = Concatenate(axis=1)([x, y])
#     z = Dense(20, activation='tanh')(z)
    
#     output = Dense(1, activation='tanh')(z)
    model = Model([context_word, target_word], cosine_sim)
    
    con_model = Model(context_word, x)
    tar_model = Model(target_word, y)
    
    return model, con_model, tar_model

In [11]:
def embeder3(input_size, embed_size):
    context_word = Input(shape=(input_size,), name="context_word")
    
    x = Dense(embed_size, activation='tanh')(context_word)
    y = Dense(embed_size, activation='tanh')(x)
    
    model = Model(context_word, y)
    em_model = Model(context_word, x)
    return model, em_model

In [12]:
def embedding_model3(input_size, output_size, embed_size):
    left_word = Input(shape=(input_size,), name="left_word")
    right_word = Input(shape=(input_size,), name="right_word")
    
    layer1 = Dense(128, activation='relu')
    layer2 = Dense(128, activation='linear')
    
    left = layer1(left_word)
    right = layer1(right_word)
    
    left = layer2(left)
    right = layer2(right)
    
    x = Concatenate(axis=1)([left, right])
    x = Dense(embed_size, activation='tanh')(x)
    
    model = Model([left_word, right_word], x)
    con_model = Model(left_word, left)
    tar_model = Model(right_word, right)
    
    return model, con_model, tar_model

In [13]:
def evaluate(final_embedding, word2int, embed_size):
    gensim = GensimWrapper(embed_size, 0, log=False)
    gensim.set_embeddings(word2int, final_embedding)
    result = gensim.evaluate()
    for key in result:
        print("{0}: {1:.2f}%".format(key, result[key]), end=' ')
    print()

In [14]:
def normalize(embeddings):
    norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
    return embeddings / norms

def normalize2(embeddings):
    maxes = np.max(np.abs(embeddings), axis=1, keepdims=True)
    return embeddings / maxes


In [15]:
words = read_file()
vocab, word2int, int2word = build_vocab(words)
int_words = words_to_ints(word2int, words)
word2freq = get_frequency(words, word2int, int2word)
char2int, int2char, char2tup, tup2char, n_consonant, n_vowel = build_charset()
ns_unigrams = ns_sample(word2freq, word2int, int2word, .75)
n_chars = 11 + 2 
n_features = len(char2int)
batch_size = 120
embed_size = 128
skip_window = 5

In [16]:
multi_gen = generate_word_images_multi(words, char2tup, batch_size, n_consonant, n_vowel)
# multi_gen = generate_word_images_multi_v2(words, char2int, char2tup, batch_size, n_consonant, n_vowel)

# [x1, x2, x3], y = next(multi_gen)
# print(y.shape)

In [44]:
multi_train, multi_enc, multi_dec = conv_model_multi(n_chars, n_consonant, n_vowel, embed_size)
adam = keras.optimizers.Nadam(0.001)
multi_train.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['acc'])
multi_train.summary()

(?, ?, 128) (?, ?, 128)
(?, ?, 128)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
root_word_input (InputLayer)    (None, 13, 50, 1)    0                                            
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 13, 50, 16)   160         root_word_input[0][0]            
__________________________________________________________________________________________________
max_pooling2d_9 (MaxPooling2D)  (None, 6, 25, 16)    0           conv2d_9[0][0]                   
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 6, 25, 8)     1160        max_pooling2d_9[0][0]            
_________________________________________________________________________

In [18]:
# multi_train2, multi_enc2, multi_dec2 = conv_model_multi_v2(n_chars, len(char2int), n_consonant, n_vowel, embed_size)
# adam = keras.optimizers.Nadam(0.001)
# multi_train2.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['acc'])
# multi_train2.summary()

In [43]:
n_batches = len(vocab) // batch_size
history = multi_train.fit_generator(multi_gen, steps_per_epoch=n_batches, epochs = 3)

Epoch 1/3


InvalidArgumentError: Incompatible shapes: [120,13] vs. [120,26]
	 [[Node: metrics_3/acc/Equal = Equal[T=DT_INT64, _device="/job:localhost/replica:0/task:0/device:GPU:0"](metrics_3/acc/ArgMax, metrics_3/acc/ArgMax_1)]]
	 [[Node: metrics_3/acc/Mean/_481 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_3647_metrics_3/acc/Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'metrics_3/acc/Equal', defined at:
  File "c:\users\leo\appdata\local\conda\conda\envs\gpu-tf\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\leo\appdata\local\conda\conda\envs\gpu-tf\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\leo\appdata\local\conda\conda\envs\gpu-tf\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launc

In [ ]:
train_vocab = vocab[:int(len(vocab) * .8)]
test_vocab = vocab[int(len(vocab) * .8):]
gen = generate_word_images_feat(vocab, word2int, char2int, sem_embed, batch_size)
# gen2 = generate_batch_image_v3(words, word2int, char2int, batch_size, skip_window)

In [ ]:
train, infenc, infdec = conv_model2(13, 13, embed_size, embed_size)
adam = keras.optimizers.Nadam(0.001)
# train.compile(optimizer=adam, loss='mse')
train.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['acc'])
# train.summary()

In [ ]:
n_batches = len(vocab) // batch_size
history = train.fit_generator(gen, steps_per_epoch=n_batches, epochs = 2)

In [ ]:
infdec.save('models/decoder.h5')

In [ ]:
def pred_embeddings(vocab, infenc):
    embeddings = np.ndarray((len(vocab), embed_size))
    i = 0
    buffer = []
    buffer_size = 10000
    embed = []
    for wi, word in enumerate(vocab):
        word = int2word[word2int[word]]
        buffer.append(word2vec(char2int, word, 13))
        embed.append(sem_embed[word2int[word]])
        if len(buffer) == buffer_size or len(vocab) - wi < buffer_size:
            buffer_np = np.stack(buffer).reshape((-1, 13, 309, 1))
            embed = np.stack(embed)
            result = infenc.predict([buffer_np, embed])
            embeddings[i:i+len(buffer)] = result
            i += len(buffer)
            buffer = []
            embed = []
            if i % (4 *buffer_size) == 0:
                print("Predicting: {0:.2f}%".format((i * 100.0 / len(vocab))))
    print("finished")
    return embeddings

embeddings = pred_embeddings(vocab, infenc)

In [22]:
def pred_embeddings_multi(vocab, encoder, char2tup):
    embeddings = np.ndarray((len(vocab), embed_size))
    i = 0
    buffer = []
    buffer_size = 10000
    for wi, word in enumerate(vocab):
        word = int2word[word2int[word]]
        convec, vowvec = word2vec_seperated(char2tup, word, n_chars, n_consonant, n_vowel)
        mat = np.concatenate([convec, vowvec], axis=1)
        buffer.append(mat)
        if len(buffer) == buffer_size or len(vocab) - wi < buffer_size:
            buffer_np = np.stack(buffer).reshape((-1, n_chars, (n_consonant + n_vowel), 1))
            result = encoder.predict(buffer_np)
            embeddings[i:i+len(buffer)] = result
            i += len(buffer)
            buffer = []
            if i % (4 *buffer_size) == 0:
                print("Predicting: {0:.2f}%".format((i * 100.0 / len(vocab))))
    print("finished")
    return embeddings

embeddings = pred_embeddings_multi(vocab, multi_enc, char2tup)

Predicting: 13.43%
Predicting: 26.86%
Predicting: 40.29%
Predicting: 53.72%
Predicting: 67.15%
Predicting: 80.58%
Predicting: 94.01%
finished


In [36]:

# for i in range(len(test_vocab)):
#     word = test_vocab[i]
#     result = decode_sequence(infdec, int2char, embeddings[word2int[word]].reshape((1, -1)))
#     result = ''.join(result).strip()#[1:-1]
#     print(word, result)
#     if i ==  10:
#         break
rand_vocab = np.random.choice(vocab, 10)
for i in range(10):
    word = rand_vocab[i]
    result = decode_multi_sequence(multi_dec, char2tup, tup2char, 
                                   embeddings[word2int[word]].reshape((1, -1)), n_consonant, n_vowel)
    result = ''.join(result).strip()#[1:-1]
    print(word, result)
    if i ==  10:
        break

ፈላልጠህ ለያይንU
ንድፍና ንንግፋ
የማይስማማው የያልያጋUU
ሳይነስ ሳይሰU
የሚያገናኘን የያያየጋጣ
ጃኮብ ድሽች
ከማወጣ ከካ*ዛ
በከፍተኛ በበችገኛ
በብረት በብተይ
አሳልፈው አላልቸU


In [24]:
embedding_normal = normalize(embeddings)

In [25]:
evaluate(embedding_normal, embed_size=embed_size, word2int=word2int)

c:\users\leo\appdata\local\conda\conda\envs\gpu-tf\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


anomaly: 68.81% semantic: 1.08% syntactic: 11.36% 


In [ ]:
np.save("results/char_embedding", embeddings)

In [ ]:
sentenses = open('data/news.txt', encoding='utf-8').read().split('*')
sentenses = [s.strip().split() for s in sentenses]


In [ ]:
model = gensim.models.Word2Vec(sentenses, 
                            size=128, 
                            iter=20, 
                            min_count=1,
                            negative=10,
                            sg=1
                            )

In [ ]:
result1 = model.accuracy('data/syntax.txt')
result2 = model.accuracy('data/semantic.txt')

In [ ]:
sem_embed = np.ndarray((len(vocab), 128))
for voc in vocab:
    if voc is not '*':
        sem_embed[word2int[voc]] = model.wv[voc]
np.save("results/word2vec_embedding", sem_embed)

In [ ]:
sem_embed_normal = normalize(sem_embed)
# emb_norm = normalize(embeddings)
# ee = sem_embed + embeddings*.1366
# full_embed = np.concatenate([5*sem_embed_normal, 2*emb_norm], axis=1)
full_embed_normal = sem_embed_normal + embedding_normal

In [ ]:
full_embed_normal = normalize(embedding_normal)

In [ ]:
evaluate(full_embed_normal, word2int, embed_size=full_embed_normal.shape[1])

In [ ]:
emu_model, emu_pred = embeder3(128, 128)
adam = keras.optimizers.Nadam(0.0001)
sgd = keras.optimizers.SGD(.01)
emu_model.compile(optimizer=adam, loss='mse')
batch_size = 500
skip_window = 5
gen4 = generate4(int_words, embeddings, word2int, batch_size, skip_window)

In [ ]:
n_batches = len(words) // batch_size
history = emu_model.fit_generator(gen4, steps_per_epoch=n_batches, epochs = 4)

In [ ]:
myembedding_norm=emu_model.predict(embedding_normal)

In [ ]:
myembeddings = normalize(myembedding_norm)
# sem_embed_normal = normalize(sem_embed)
ee = myembeddings #+ sem_embed_normal
evaluate(ee, word2int, embed_size=myembeddings.shape[1])

In [ ]:
# min = 1
# b = embedding_normal[0]
# for i in range(len(embedding_normal)):
#     a = embedding_normal[i]
#     d = a.dot(b)
#     if d < min:
#         print(d)
#         min = d
# embedding_normal[0]

In [ ]:
def get_batch_words(words, start, length):
    if start + length > len(words):
        end = start + length - len(words)
        return words[start:] + words[0:end], end
    else:
        end = start + length
        return words[start:end], end

def get_context_words(words, start, length):
    if start + length > len(words):
        start = 0
    end = start + length
    return words[start:end], start


In [ ]:
def generate(data, embeds, word2int, int2word, unigrams, batch_size, skip_window):
    embed_szie = embeds.shape[1]
    assert batch_size % skip_window == 0
    ci = skip_window  # current_index
    batch_y = np.ones(shape=(batch_size, 1), dtype=np.float32)
    while True:
        batch_inputs = np.ndarray(shape=(batch_size, embed_size), dtype=np.float32)
        batch_labels = np.ndarray(shape=(batch_size, embed_size), dtype=np.float32)
        batch_index = 0
        shuffle_index = np.random.shuffle(np.arange(batch_size))
        for batch_index in range(0, batch_size, skip_window * 2):  # fill the batch inputs
            context = data[ci - skip_window:ci + skip_window + 1]
            # remove the target from context words
            target = context.pop(skip_window)
            # context = random.sample(context, skip_window * 2)
            word_index = 0
            for b in range(batch_index, batch_index + skip_window * 2):
                con_vec = embeds[word2int[context[word_index]]]
                target_vec = embeds[word2int[target]]
                batch_inputs[b] = con_vec
                batch_labels[b] = target_vec
                word_index += 1

            ci += 1
        if len(data) - ci - skip_window < batch_size:
            ci = skip_window
        for ri  in range(0, batch_size, 2):
            batch_labels[ri] = embeds[np.random.randint(len(embeds))]
            batch_y[ri][0] = batch_labels[ri].dot(batch_inputs[ri])
#         print(batch_labels.shape)
#         batch_labels = batch_labels[shuffle_index].reshape((-1, 128))
# #         print(batch_labels.shape)
#         batch_inputs = batch_inputs[shuffle_index].reshape((-1, 128))
#         batch_y = batch_y[shuffle_index].reshape((-1, 1))
        yield [batch_inputs, batch_labels], batch_y


In [ ]:
def generate2(data, embeds, word2int, batch_size, skip_window):
    embed_size = embeds.shape[1]
    assert batch_size % skip_window == 0
    ci = skip_window  # current_index
    input_width = embed_size * 2 * skip_window
    while True:
        batch_inputs = np.ndarray(shape=(batch_size, input_width), dtype=np.float32)
        batch_labels = np.ndarray(shape=(batch_size, embed_size), dtype=np.float32)
        batch_index = 0
        for batch_index in range(batch_size):  # fill the batch inputs
            context = data[ci - skip_window:ci + skip_window + 1]
            target = context.pop(skip_window)
#             print(context, target)
            context_vec = []
            target_vec = embeds[word2int[target]]
            for word in context:
                con_vec = embeds[word2int[word]]
                context_vec.append(con_vec)
            context_vec = np.hstack(context_vec)
#             batch_inputs[batch_index] = context_vec
#             batch_labels[batch_index] = target_vec
            
            ci += 1
        if len(data) - ci - skip_window < batch_size:
            ci = skip_window
        yield batch_inputs, batch_labels


In [ ]:
def generate3(data, embeds, word2int, batch_size, skip_window):
    embed_size = embeds.shape[1]
    ci = 0  # current_word_index
    input_width = embed_size * 2 * skip_window
    while True:
        batch_inputs_left = np.ndarray(shape=(batch_size, embed_size), dtype=np.float32)
        batch_inputs_right = np.ndarray(shape=(batch_size, embed_size), dtype=np.float32)
        batch_labels = np.ndarray(shape=(batch_size, embed_size), dtype=np.float32)
        batch_index = 0
        for batch_index in range(batch_size):  # fill the batch inputs
            context, ci = get_context_words(data, ci, 3)
            ci = ci + 1
            left_word_vec = embeds[word2int[context[0]]]
            target_vec = embeds[word2int[context[1]]]
            right_word_vec = embeds[word2int[context[2]]]
            batch_inputs_left[batch_index] = left_word_vec
            batch_inputs_right[batch_index] = right_word_vec
            batch_labels[batch_index] = target_vec

        yield [batch_inputs_left, batch_inputs_right], batch_labels


In [ ]:
def generate4(data, embeds, word2int, batch_size, skip_window):
    embed_size = embeds.shape[1]
    ci = 0  # current_word_index
    input_width = embed_size
    while True:
        batch_inputs = np.ndarray(shape=(batch_size, embed_size), dtype=np.float32)
        batch_labels = np.ndarray(shape=(batch_size, embed_size), dtype=np.float32)
        for batch_index in range(0, batch_size, skip_window):  # fill the batch inputs
            context, ci = get_context_words(data, ci, skip_window * 2 + 1)
            ci = ci + 1
            target = context.pop(skip_window)
            context = np.random.choice(context, skip_window)
            context_vec = embeds[context[0]]
            target_vec = embeds[context[1]]
            
            batch_inputs[batch_index:batch_index +
                             skip_window] = embeddings[context]
            batch_labels[batch_index:batch_index +
                             skip_window] = embeddings[target]

        yield batch_inputs, batch_labels


In [ ]:
gen3 = generate3(words, embedding_normal, word2int, batch_size, 3)

In [ ]:
window = 3
semantic_batch_size = 120
input_size = 128
# gg =  generate2(words, embeds_norm, word2int, batch_size=semantic_batch_size, skip_window=window) 

In [ ]:
train_model, con_model, tar_model = embedding_model3(input_size, 128, embed_size)
adam = keras.optimizers.Nadam(0.0001)
train_model.compile(optimizer=adam, loss="mse")
train_model.summary()

In [ ]:
n_batches = len(words) // semantic_batch_size
history = train_model.fit_generator(gen3, steps_per_epoch=n_batches, epochs = 2)

In [ ]:
g = generate(words, embedding_normal, word2int, int2word, ns_unigrams, batch_size=semantic_batch_size, skip_window=3)
[a, b], y = next(g)
print(y)

In [ ]:
input_size = 128#window * 2* embed_size
# em_train, em_out = embedding_model(input_size, 128, embed_size)
# adam = keras.optimizers.Nadam(lr=0.002)
# em_train.compile(optimizer=adam, loss='mean_squared_error')
# em_train.summary()

In [ ]:
train_model, con_model, tar_model = embedding_model2(input_size, 128, embed_size)
adam = keras.optimizers.SGD(0.001)
train_model.compile(optimizer=adam, loss="mse", metrics=['mse', 'acc'])
train_model.summary()

In [ ]:
n_batches = len(words) // semantic_batch_size
history = train_model.fit_generator(g, steps_per_epoch=n_batches, epochs = 2)

In [ ]:
context_vecs = []
for i_word in range(len(vocab)):
    word = int2word[i_word]
    context_vecs.append(embedding_normal[word2int[word]])
context_vecs = np.stack(context_vecs)
context_embed = con_model.predict(context_vecs)
# target_embed = tar_model.predict(context_vecs)
# em = context_embed + target_embed

In [ ]:
em_normal = normalize(full_embed)
evaluate(em_normal, word2int, embed_size=em_normal.shape[1])

In [ ]:
utils = Utils(word2int,int2word, embedding_normal)
# v = -em_normal[word2int['ገንዘብ']] + em_normal[word2int['ብር']]
# dots = em_normal.dot(v).flatten()
# int2word[np.argmax(dots)]
utils.sorted_sim("ዶላር")
# utils.sorted_sim("ብር")

In [ ]:
con_input = [
    embedding_normal[word2int['ነበር']].reshape((1, 128)),
    embedding_normal[word2int['ነው']].reshape((1, 128)),
]
vec = train_model.predict(con_input).flatten()
print(embedding_normal[word2int['ነበር']].dot(embedding_normal[word2int['ነው']]))
int2word[np.argmax(embedding_normal.dot(vec))]

In [ ]:
min = 1
b = em_normal[0]
for i in range(len(em_normal)):
    a = em_normal[i]
    d = a.dot(b)
    if d < min:
        print(d)
        min = d
# embedding_normal[0]

In [ ]:
# semantic = em_out.predict(embeds_norm)
# gensim = GensimWrapper(embed_size, 0, log=True)
# embeds = embeds.reshape((-1, 128))
# norms = np.linalg.norm(semantic, axis=1, keepdims=True)
# semantic_norm = semantic / norms
vecs = []
discovered = {}
for i in range(window, len(words) - window):
    context = words[i - window: i + window + 1]
    target = context.pop(window)
    if target not in discovered:
        discovered[target] = len(discovered)
        c_vec = []
        for cword in context:
            vec = embeds_norm[word2int[cword]]
            c_vec.append(vec)
        context_vec = np.hstack(c_vec)
        vecs.append(context_vec)
    if len(discovered) == len(vocab):
        print("discovered")
        break
    
semantic = np.stack(vecs).reshape(-1, input_size)
print(len(vecs), embeds_norm.shape)
assert semantic.shape[0] == embeds_norm.shape[0]
# semantic = em_out.predict(embeds_norm)
# gensim = GensimWrapper(embed_size, 0, log=True)
# embeds = embeds.reshape((-1, 128))
# norms = np.linalg.norm(semantic, axis=1, keepdims=True)
# semantic_norm = semantic / norms
